# 0. Configuración de librerias y requisitos previos

In [85]:
"""
Se importan las librerías necesarias para el procesamiento de texto y manejo de archivos.

- nltk.tokenize.regexp_tokenize: para tokenización basada en expresiones regulares.
- nltk.stem.SnowballStemmer: para aplicar stemming en inglés.

- xml.etree.ElementTree: para parseo de archivos en formato XML.

- collections: para estructuras de datos con valores por defecto y operaciones de conteo.

- os: para operaciones relacionadas con el sistema de archivos.

- typing: para anotaciones de tipo.

- math: para operaciones matemáticas.
"""

from nltk.tokenize import regexp_tokenize
from nltk.stem import SnowballStemmer

import xml.etree.ElementTree as ET

from collections import Counter, defaultdict

from typing import List, Dict, Union, Set

import math

import os

In [86]:
"""
Carga de stopwords desde un archivo local.

- Evita la descarga de NLTK y permite usar una lista personalizada de palabras vacías.
- Se construye el conjunto `stop_words` para filtrar palabras vacías durante el preprocesamiento.
"""

# NO cambie esta ruta
path_stopwords = "../data/stopwords/english"

def load_stopwords(file_path: str) -> Set[str]:
    
    """
    Carga stopwords desde un archivo de texto plano.

    Cada línea del archivo debe contener una palabra. La función convierte
    todas las palabras a minúsculas y elimina líneas vacías.

    Parámetros
    ----------
    file_path : str
        Ruta del archivo de stopwords.

    Retorna
    -------
    Set[str]
        Conjunto de palabras que deben considerarse stopwords.
    """
    
    with open(file_path, "r", encoding = "utf-8") as file:
        stop_words = {line.strip().lower() for line in file if line.strip()}
    return stop_words

stop_words = load_stopwords(path_stopwords)

In [87]:
"""
Configuración del stemmer y definición del patrón de tokenización.

- Se inicializa `SnowballStemmer("english")` para reducir las palabras a su raíz 
  durante el preprocesamiento.
- Se define la expresión regular `pattern` que controla cómo se segmenta el texto
  en tokens. Incluye reglas para:
    * Abreviaturas (e.g., U.S.A.)
    * Palabras con guiones internos (e.g., non-euclidean)
    * Monedas, números y porcentajes
    * Puntos suspensivos (...)
    * Signos de puntuación como tokens separados
"""

stemmer = SnowballStemmer("english")

pattern = r'''(?x)              
    (?:[A-Z]\.)+[A-Z]?          # abreviaturas, e.g. U.S.A.
  | [A-Za-z]+(?:-[A-Za-z]+)*    # palabras con guiones internos, e.g. non-euclidean
  | \$?\d+(?:\.\d+)?%?          # monedas, números, porcentajes
  | \.\.\.                      # puntos suspensivos
  | [][.,;"'?():_`-]            # puntuación como tokens separados
'''

# 1. Preprocesamiento del texto

In [88]:
def preprocess(text: str) -> List[str]:
    
	"""
    Preprocesa un texto aplicando tokenización, normalización, eliminación de stopwords y stemming.

    El proceso de preprocesamiento transforma un texto crudo en una lista de tokens
    limpios y estandarizados, listos para tareas de recuperación de información o NLP.

    Pasos:
        1. Tokenización: se divide el texto en palabras usando la expresión regular `pattern`.
        2. Normalización: se convierten todos los tokens a minúsculas.
        3. Eliminación de stopwords: se filtran palabras vacías presentes en `stop_words`.
        4. Stemming: se aplica `SnowballStemmer` para reducir los tokens a su raíz.

    Parámetros
    ----------
    text : str
        Texto de entrada a procesar.

    Retorna
    -------
    List[str]
        Lista de tokens preprocesados.
    """
      
	# Generación de tokens
	tokens = regexp_tokenize(text, pattern)
	# Normalización 
	tokens = [token.lower() for token in tokens]
	# Eliminación de palabras de parada
	tokens = [token for token in tokens if token not in stop_words]
	# Stemming 
	tokens = [stemmer.stem(token) for token in tokens]
	return tokens

In [89]:
"""
Ejemplo de ejecución del preprocesamiento de texto.

Se procesa un texto de ejemplo que incluye números, signos de puntuación y abreviaturas. 
"""

example_text = "Se deben invertir $12.000 millones en D.I.A ..."

print("--------** preprocess **--------\n")
print("Parametros:\n")
print("- Texto de entrada = ", example_text)
print("\nResultado:\n")
print("- Tokens procesados = ", preprocess(example_text))


--------** preprocess **--------

Parametros:

- Texto de entrada =  Se deben invertir $12.000 millones en D.I.A ...

Resultado:

- Tokens procesados =  ['se', 'deben', 'invertir', '$12.000', 'millon', 'en', 'd.i.a', '...']


# 2. Carga de documentos

In [90]:
"""
Ruta de los documentos a procesar.

- `path_docs` indica la carpeta donde se encuentran los archivos de texto crudos.
- Cambie esta ruta según la ubicación de sus documentos en su sistema.
"""

# Cambie esta ruta según la ubicación de los archivos en su sistema
path_docs = "../data/docs-raw-texts"

In [91]:
def load_documents(path_docs: str) -> Dict[str, List[str]]:

	"""
    Carga y preprocesa los documentos de una carpeta específica.

    La función recorre todos los archivos con extensión `.naf` en la ruta
    especificada, extrae el ID del documento, el título y el contenido crudo,
    concatena título y contenido, y aplica la función `preprocess` para 
    obtener los tokens preprocesados de cada documento.

    Parámetros
    ----------
    path_docs : str
        Ruta de la carpeta que contiene los archivos `.naf` a procesar.
        Cambie esta ruta según la ubicación de sus documentos.

    Retorna
    -------
    Dict[str, List[str]]
        Diccionario donde las claves son los `doc_id` de cada documento y
        los valores son listas de tokens preprocesados.
    """
	
	docs = {}
	for file_name in os.listdir(path_docs):
		# Solo se revisan los archivos con extensión correcta
		if not file_name.endswith(".naf"):
			continue
		tree = ET.parse(os.path.join(path_docs, file_name))
		root = tree.getroot()
		# Id recuperado desde el atributo pupblicId
		doc_id = root.find("nafHeader/public").attrib["publicId"]
		# Titulo desde el atributo title
		title = root.find("nafHeader/fileDesc").attrib.get("title", "")
		# Contenido desde <raw>
		raw_element = root.find("raw")
		content = raw_element.text if raw_element is not None else ""
		# concatenar titulo + contenido para preprocesar
		tokens = preprocess(title +  " " + content)
		docs[doc_id] = tokens
	return docs

In [92]:
"""
Ejemplo de carga y preprocesamiento de documentos.

Se utiliza la función `load_documents` para cargar todos los archivos `.naf`
de la carpeta especificada en `path_docs`. Cada documento se preprocesa y se almacena
en un diccionario donde la clave es el `doc_id` y el valor es la lista de tokens.

Luego se muestra el contenido preprocesado de un documento específico
con ID "d006".
"""

documents = load_documents(path_docs)
example_id = "d006"

print("--------** load documents **--------\n")
print("Parametros:\n")
print("- Ruta de los documentos = ", path_docs)
print("\nResultado:\n")
print("- Documentos cargados = ", len(documents))
print("- Ejemplo documento " + example_id + " = ", documents[example_id])

--------** load documents **--------

Parametros:

- Ruta de los documentos =  ../data/docs-raw-texts

Resultado:

- Documentos cargados =  331
- Ejemplo documento d006 =  ['eugenio', 'beltrami', 'non-euclidian', 'geometri', 'eugenio', 'beltrami', 'non-euclidian', 'geometri', '.', 'eugenio', 'beltrami', '(', '1835', '-', '1900', ')', '.', 'novemb', '16', ',', '1835', ',', 'italian', 'mathematician', 'eugenio', 'beltrami', 'born', '.', 'notabl', 'work', 'concern', 'differenti', 'geometri', 'mathemat', 'physic', '.', 'work', 'note', 'especi', 'clariti', 'exposit', '.', 'first', 'prove', 'consist', 'non-euclidean', 'geometri', 'model', 'surfac', 'constant', 'curvatur', ',', 'pseudospher', '.', 'eugenio', 'beltrami', 'born', 'cremona', 'lombardi', ',', 'part', 'austrian', 'empir', ',', 'part', 'itali', '.', 'son', 'artist', 'paint', 'miniatur', ',', 'young', 'eugenio', 'certain', 'inherit', 'artist', 'talent', 'famili', ',', 'case', 'addit', 'mathemat', 'talent', 'would', 'acquir', ',', 'm

# 3. Construcción del índice invertido

In [93]:
def build_inverted_index(docs: Dict[str, List[str]]) -> Dict[str, Dict[str, Union[int, List[str]]]]:
    
	"""
    Construye un índice invertido a partir de un conjunto de documentos preprocesados.

    El índice invertido permite recuperar rápidamente qué documentos contienen
    cada token. Para cada token, se almacena:
        - 'df': la frecuencia de documentos (número de documentos que contienen el token).
        - 'postings': la lista ordenada de IDs de documentos donde aparece el token.

    Parámetros
    ----------
    docs : Dict[str, List[str]]
        Diccionario de documentos preprocesados. Las claves son `doc_id` y los
        valores son listas de tokens obtenidos con la función `preprocess`.

    Retorna
    -------
    Dict[str, Dict[str, Union[int, List[str]]]]
        Diccionario donde cada clave es un token y su valor es un diccionario
        con la frecuencia de documentos ('df') y la lista de postings ('postings').
    """
     
	inverted = defaultdict(set)
	for doc_id, tokens in docs.items():
		for token in tokens:
			inverted[token].add(doc_id)
	# Convertir sets a listas ordenadas
	for token in inverted:
		inverted[token] = {"df" : len(inverted[token]), "postings" : sorted(inverted[token])}
	return inverted

In [94]:
"""
Ejemplo de construcción del índice invertido.

Se utiliza la función `build_inverted_index` para generar un índice invertido
a partir del diccionario `documents` preprocesados. El índice permite consultar
rápidamente en qué documentos aparece cada token y su frecuencia de documentos (df).

Luego se muestra un ejemplo de token presente en el índice.
"""

inverted_index = build_inverted_index(documents)
example_token = "play"

print("--------** build inverted index **--------\n")
print("Parámetros:\n")
print("- Documentos = ", documents)
print("\nResultado:\n")
print("- Número de tokens en el índice = ", len(inverted_index))
print(f"- Ejemplo token " + example_token + " = ", inverted_index.get(example_token))

--------** build inverted index **--------

Parámetros:

- Documentos =  {'d210': ['joseph', 'weizenbaum', 'famous', 'eliza', 'joseph', 'weizenbaum', 'famous', 'eliza', '.', 'joseph', 'weizenbaum', '(', '1923', '-', '2008', ')', 'photo', ':', 'ulrich', 'hansen', '.', 'januari', '8', ',', '1923', ',', 'comput', 'scientist', 'joseph', 'weizenbaum', ',', 'pioneer', 'natur', 'languag', 'process', 'artifici', 'intellig', ',', 'later', 'becam', 'one', 'artifici', 'intellig', 'lead', 'critic', ',', 'born', '.', '1966', 'publish', 'simpl', 'program', 'name', 'eliza', ',', 'involv', 'user', 'convers', 'bore', 'strike', 'resembl', 'one', 'psychologist', '.', 'joseph', 'weizenbaum', 'born', 'berlin', 'jewish', 'parent', 'januari', '8', ',', '1923', '.', 'abl', 'escap', 'nazi', 'germani', 'januari', '1936', ',', 'emigr', 'famili', 'unit', 'state', ',', 'start', 'studi', 'mathemat', 'wayn', 'state', 'univers', 'detroit', '1941', '.', 'howev', ',', 'studi', 'interrupt', 'war', ',', 'serv', 'militari

# 4. Representación vectorial Tf-IDf

In [95]:
def build_tf_index(documents: Dict[str, List[str]]) -> Dict[str, Dict[str, int]]:
    
	"""
    Construye un índice de frecuencia de términos (TF) a partir de los documentos preprocesados.

    La función recorre todos los documentos proporcionados y cuenta la frecuencia de
    cada token dentro de cada documento. El resultado es un diccionario donde cada
    documento tiene un subdiccionario que representa la frecuencia de cada término.

    Parámetros
    ----------
    documents : Dict[str, List[str]]
        Diccionario donde las claves son los `doc_id` de los documentos y los valores
        son listas de tokens preprocesados de cada documento, tal como devuelve
        la función `load_documents`.

    Retorna
    -------
    Dict[str, Dict[str, int]]
        Diccionario donde las claves son los `doc_id` de los documentos y los valores
        son diccionarios que mapean cada token a su frecuencia dentro del documento.
        Por ejemplo: { "doc1": {"palabra1": 3, "palabra2": 5}, ... }.
    """
     
	tf_index = {}
	# Conteo de la frecuencia de los términos dentro de cada documento
	for doc_id, tokens in documents.items():
		tf_index[doc_id] = dict(Counter(tokens))
	return tf_index

In [96]:
"""
Ejemplo de construcción del índice de frecuencia de términos (TF).

Se utiliza la función `build_tf_index` para generar un índice que contiene
la frecuencia de cada token dentro de cada documento preprocesado en `documents`.

Luego se muestra el subdiccionario correspondiente a un documento específico
con ID "d006", mostrando cuántas veces aparece cada token en ese documento.
"""

tf_index = build_tf_index(documents)
example_id = "d006"

print("--------** build TF index **--------\n")
print("Parametros:\n")
print("- documentos = ", documents)
print("\nResultado:\n")
print("- Índice TF generado = ", tf_index)
print("- Ejemplo de frecuencias del documento " + example_id + " = ", tf_index[example_id])


--------** build TF index **--------

Parametros:

- documentos =  {'d210': ['joseph', 'weizenbaum', 'famous', 'eliza', 'joseph', 'weizenbaum', 'famous', 'eliza', '.', 'joseph', 'weizenbaum', '(', '1923', '-', '2008', ')', 'photo', ':', 'ulrich', 'hansen', '.', 'januari', '8', ',', '1923', ',', 'comput', 'scientist', 'joseph', 'weizenbaum', ',', 'pioneer', 'natur', 'languag', 'process', 'artifici', 'intellig', ',', 'later', 'becam', 'one', 'artifici', 'intellig', 'lead', 'critic', ',', 'born', '.', '1966', 'publish', 'simpl', 'program', 'name', 'eliza', ',', 'involv', 'user', 'convers', 'bore', 'strike', 'resembl', 'one', 'psychologist', '.', 'joseph', 'weizenbaum', 'born', 'berlin', 'jewish', 'parent', 'januari', '8', ',', '1923', '.', 'abl', 'escap', 'nazi', 'germani', 'januari', '1936', ',', 'emigr', 'famili', 'unit', 'state', ',', 'start', 'studi', 'mathemat', 'wayn', 'state', 'univers', 'detroit', '1941', '.', 'howev', ',', 'studi', 'interrupt', 'war', ',', 'serv', 'militari', 'me

In [97]:
def compute_tdidf_vector(doc_id: str, tf_index: Dict[str, Dict[str, int]], inverted_index: Dict[str, Dict[str, Union[int, List[str]]]], num_documents: int) -> Dict[str, float]:
	
	"""
	Construye el vector TF-IDF de un documento específico.

	La función recibe el identificador de un documento junto con el índice de 
	frecuencias (TF), el índice invertido y el número total de documentos. 
	Para cada token del documento, se calcula el peso TF-IDF aplicando la 
	fórmula:

		TF-IDF = log10(frecuencia + 1) * log10(N / df)

	donde N es el número total de documentos y df es la frecuencia de 
	documentos en que aparece el token.

	Parámetros
	----------
	doc_id : str
		Identificador del documento para el cual se construirá el vector TF-IDF.
	tf_index : Dict[str, Dict[str, int]]
		Índice de frecuencias de términos por documento. Las claves son los 
		`doc_id` y los valores son diccionarios de tokens y sus frecuencias.
	inverted_index : Dict[str, Dict[str, Union[int, List[str]]]]
		Índice invertido que contiene, para cada token, su frecuencia de 
		documento (`df`) y la lista de documentos en los que aparece.
	num_documents : int
		Número total de documentos en la colección.

	Retorna
	-------
	Dict[str, float]
		Vector TF-IDF representado como un diccionario disperso, donde las claves 
		son los tokens del documento y los valores son los pesos TF-IDF 
		correspondientes.
	"""

	# Frecuencia de términos del documento
	doc_tf = tf_index[doc_id]
	# Se modela el vector como un diccionario debido a que es muy dispero
	tf_idf = {}
	for token, frequency in doc_tf.items():
		# Para evitar errores, se revisa que el token este en el índice invertido
		if token in inverted_index:
			log_tf = math.log10(frequency + 1)
			idf = math.log10(num_documents / inverted_index[token]["df"])
			tf_idf[token] = log_tf * idf
	return tf_idf

In [ ]:
"""
Ejemplo de construcción del vector TF-IDF de un documento.

Se utiliza la función `compute_tdidf_vector` para calcular el vector TF-IDF
del documento con ID "d006". El cálculo se realiza a partir del índice de 
frecuencias (TF), el índice invertido y el número total de documentos.

El resultado es un diccionario disperso donde las claves son los tokens del
documento y los valores son sus respectivos pesos TF-IDF.
"""

example_id = "d006"
num_documents = len(documents)

tfidf_vector = compute_tdidf_vector(example_id, tf_index, inverted_index, num_documents)

print("--------** compute TF-IDF vector **--------\n")
print("Parámetros:\n")
print("- Documento = ", example_id)
print("- Número total de documentos = ", num_documents)
print("\nResultado:\n")
print("- Vector TF-IDF del documento " + example_id + " (primeros 10 términos):\n")

# Mostrar solo los primeros 10 términos para mayor legibilidad
for token, weight in list(tfidf_vector.items())[:10]:
    print(f"  {token}: {weight:.4f}")


--------** compute TF-IDF vector **--------

Parámetros:

- Documento =  d006
- Número total de documentos =  331

Resultado:

- Vector TF-IDF del documento d006 (primeros 10 términos):

  eugenio: 1.8751018393917862
  beltrami: 2.9337375242203336
  non-euclidian: 1.4277907382552266
  geometri: 1.4872326568185101
  .: 0.0
  (: 0.05082688261488847
  1835: 0.70539290946943
  -: 0.14678176857950007
  1900: 0.41352489242969576
  ): 0.05082688261488847
